# PyCity School Analysis

This analysis evaluates district-wide standardized reading and math test scores. The data includes specific schools, school type, school budget and school size (number of students). This report groups these data points by school, and evaluates test scores averages (math and reading) at each school. The report also sub-divides the data by school type, size and budget for targeted analysis. 
 
 
 Findings:
 
 - Student outcomes tend to decline in schools with higher per-student spending. Students performed best in schools with per-    capita budgets < 585, and students showed the worst outcomes at schools with per-capita budgets from 645-680.
 
 - Student outcomes in math and reading are similar in small (<1000) and medium (1000 to 2000) schools, but decline noticeably      in large (2000-5000) schools. 
 
 - Students in charter schools so better outcomes in math and reading than students enrolled in district schools.

In [25]:
#import dependencies (1)
import pandas as pd
from pathlib import Path

In [26]:
#creating initial dataframes, reading in and then converting csv data into a pandas df (1)
schools_df = pd.read_csv(Path("Resources/schools_complete.csv"))
students_df = pd.read_csv(Path("Resources/students_complete.csv"))
#sorting schools_df by school name, alphabetically (6)(8). The index is reset.
schools_df = schools_df.sort_values(by='school_name').reset_index()

# Districut Summary

In [27]:
#calculate passing rates for math, reading and total. using 70/100 as benchmark for 'passing'. (2)

math_passing = (students_df["math_score"].loc[students_df["math_score"]>=70].count() / students_df["math_score"].count()) * 100

reading_passing = (students_df["reading_score"].loc[students_df["reading_score"]>=70].count() / students_df["reading_score"].count()) * 100

#idenitying rows where reading score and math score are both >= 70, and calculating a percentage (3)
total_passing = (((students_df["reading_score"]>=70) & (students_df["math_score"]>=70)).sum() / students_df["reading_score"].count()) * 100


In [28]:
#storing all target values and categories in a dictionary. (16)
district_summary_data = {
    "Category": ["Total Schools", 
                 "Total Students", 
                 "Total Budget",
                 "Average Math Score",
                 "Average Reading Score",
                 "Math Passing %",
                 "Reading Passing %",
                 "Total Passing %"
                ],
    "Value": [schools_df["school_name"].count(),
              students_df["student_name"].count(),
              schools_df["budget"].sum(),
              round(students_df["math_score"].mean(),2),
              round(students_df["reading_score"].mean(),2),
              round(math_passing, 2),
              round(reading_passing, 2),
              round(total_passing, 2)
            ]
    }

In [29]:
#converting dictionary object into a dataframe. (4)
district_summary = pd.DataFrame(district_summary_data)

# Formatting specific cell values (17)
district_summary.iloc[0,1] = "{:,.0f}".format(district_summary.iloc[0,1])
district_summary.iloc[1,1] = "{:,.0f}".format(district_summary.iloc[1,1])
district_summary.iloc[2,1] = "${:,.2f}".format(district_summary.iloc[2,1])

#setting the category values as index values (5)
district_summary = district_summary.set_index('Category', drop=True)

district_summary.head(20)

,Value
Category,
Total Schools,15
Total Students,"39,170"
Total Budget,"$24,649,428.00"
Average Math Score,78.99
Average Reading Score,81.88
Math Passing %,74.98
Reading Passing %,85.81
Total Passing %,65.17


# School Summary

In [30]:
#creating dataframes that contain average scores by school, using .groupby() and .mean() (6). 
# resetting the index of each dataframe with reset_index() (7), to match the formatting of schools_df.
mathAVG = (pd.DataFrame(students_df.groupby(["school_name"]).mean()).reset_index())[["school_name","math_score"]]
readAVG = (pd.DataFrame(students_df.groupby(["school_name"]).mean()).reset_index())[["school_name","reading_score"]]

In [31]:
#creating a dictionary of desired values to pass into pd.DataFrame
school_summary_data = {
    "school_name": schools_df["school_name"],
    "School Type": schools_df["type"],
    "Student Count": schools_df["size"],
    "Total Budget ($)": schools_df["budget"],
    "Per-Student Budget ($)": (schools_df["budget"]/schools_df["size"]),
    "Average Math Score": mathAVG["math_score"],
    "Average Reading Score": readAVG["reading_score"]  
    }

In [32]:
#creating the first iteration of the school summary dataframe
school_summary_df = pd.DataFrame(school_summary_data)


In [33]:
#find the total number of students per school, creating new dataframe. This will be referenced in code below.
school_pop = pd.DataFrame(students_df.groupby("school_name").size()).reset_index()
school_pop = school_pop.rename(columns = {0:"Total Students"})


In [34]:
#code block to determing the count and percentage of students passing math, by school
#adding this data to overall school summary dataframe (school_summary_df).

#find the number of students passing math
passing_math_count = students_df.loc[(students_df['math_score']>=70)]
#grouping loc output and creating a new dataframe. counting the size of each group with .size() (11).
passing_math_count = pd.DataFrame(passing_math_count.groupby("school_name").size()).reset_index()
#rename columns (9)
passing_math_count = passing_math_count.rename(columns = {0:"Number of Students Passing Math"})


#merging two dataframes on shared series 'school_name' (10)
math_merge = pd.merge(passing_math_count,school_pop, on="school_name")
#creating a new series
math_merge["Passing Math %"] = (math_merge["Number of Students Passing Math"] / \
                                math_merge["Total Students"]) * 100


#creating simplified dataframe for merging with school_summary_df
math_merge = math_merge[["school_name", "Number of Students Passing Math", "Passing Math %"]]

school_summary_df1 = pd.merge(school_summary_df, math_merge, on="school_name")

In [35]:
#code block to determing the percentage of students passing reading, by school. this code mimics the math block above.
#adding this data to school summary data.

#find the number of students passing reading
passing_read_count = students_df.loc[(students_df['reading_score']>=70)]
#grouping loc output and creating a new dataframe. counting the size of each group with size() (11).
passing_read_count = pd.DataFrame(passing_read_count.groupby("school_name").size()).reset_index()
passing_read_count = passing_read_count.rename(columns = {0:"Number of Students Passing Reading"}) #(9)

#merging two dataframe on shared series 'school_name' (10)
#using school_pop dataframe calculated in cell above, which containts total population per school
read_merge = pd.merge(passing_read_count,school_pop, on="school_name")
#creating a new series
read_merge["Passing Reading %"] = (read_merge["Number of Students Passing Reading"] / \
                                read_merge["Total Students"]) * 100

#simplifying dataframe to target columns
read_merge = read_merge[["school_name", "Number of Students Passing Reading", "Passing Reading %"]]


#merging math data into summary dataframe
school_summary_df2 = pd.merge(school_summary_df1, read_merge, on="school_name")


In [36]:
#code block to determing the percentage of students passing reading AND math, by school. this code mimics the math block above.
#adding this data to school summary data.

#find the number of students passing reading AND passing math
passing_total_count = students_df.loc[((students_df['reading_score']>=70) & (students_df['math_score']>=70)),:]
#grouping loc output and creating a new dataframe. counting the size of each group with size() (11).
passing_total_count = pd.DataFrame(passing_total_count.groupby("school_name").size()).reset_index()
passing_total_count = passing_total_count.rename(columns = {0:"Number of Students Passing Overall"}) #(9)

#merging two dataframe on shared series 'school_name' (10)
#using school_pop dataframe calculated in cell above, which containts total population per school
total_merge = pd.merge(passing_total_count,school_pop, on="school_name")
total_merge.head(20)


#creating a new series
total_merge["Total Passing %"] = (total_merge["Number of Students Passing Overall"] / \
                                total_merge["Total Students"]) * 100
#simplifying dataframe to target columns
total_merge = total_merge[["school_name", "Number of Students Passing Overall", "Total Passing %"]]


#merging math data into summary dataframe
school_summary_df3 = pd.merge(school_summary_df2, total_merge, on="school_name")



In [37]:
#cleaning and formatting the school summary dataframe
#creating final dataframe
per_school_summary = school_summary_df3[["school_name", "School Type", "Student Count", 
                                        "Total Budget ($)","Per-Student Budget ($)", 
                                        "Average Math Score", "Average Reading Score",
                                        "Number of Students Passing Math","Passing Math %",
                                         "Number of Students Passing Reading","Passing Reading %",
                                         "Number of Students Passing Overall", "Total Passing %"]]
#modifying column names
per_school_summary = per_school_summary.rename(columns={"school_name": "School Name",
                                                        "Student Count": "Total Student Count",
                                                           "Passing Math %": "Passing Math (%)",
                                                           "Passing Reading %": "Passing Reading (%)",
                                                           "Total Passing %": "Total Passing (%)"})

#creating a copy of the final summary, to modify later for additional analysis (14)
school_summary_final_copy = per_school_summary.copy()
per_school_summary.head(15)

,School Name,School Type,Total Student Count,Total Budget ($),Per-Student Budget ($),Average Math Score,Average Reading Score,Number of Students Passing Math,Passing Math (%),Number of Students Passing Reading,Passing Reading (%),Number of Students Passing Overall,Total Passing (%)
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,3318,66.680064,4077,81.933280,2719,54.642283
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,1749,94.133477,1803,97.039828,1697,91.334769
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,1946,65.988471,2381,80.739234,1569,53.204476
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,1871,68.309602,2172,79.299014,1487,54.289887
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,1371,93.392371,1426,97.138965,1330,90.599455
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,3094,66.752967,3748,80.862999,2481,53.527508
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,395,92.505855,411,96.252927,381,89.227166
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,1916,65.683922,2372,81.316421,1561,53.513884
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,3145,66.057551,3867,81.222432,2549,53.539172
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,910,94.594595,923,95.945946,871,90.540541


Note regarding module challenge instrcutions. 

In the 'requirements' section, the challenge document calls for the following:
- "Calculate the number of schools with math scores of 70 or higher (2 points)"
- "Calculate the number of schools with reading scores of 70 or higher (2 points)"
- Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher (2 points)

This instruction is unclear, as this would be a single value in each case, which does not fit within the dataframe. Instead, I have provided the following:
- "Calculate the number of studnets, per school, with math scores of 70 or higher (2 points)"
- "Calculate the number of students, per school, with reading scores of 70 or higher (2 points)"
- "Use the provided code to calculate the the number of students, per school, that passed both math and reading with scores of 70 or higher (2 points)

## Highest-Performing Schools by Percentage of Overall Passing

In [38]:
#organizing the summary data frame by 'total passing (%)', in descending order.
top_schools = school_summary_final_copy.sort_values(by="Total Passing (%)",ascending=False)
#renaming column, to conform with challenge requirements
top_schools = top_schools.rename(columns={"Total Passing (%)": "% Overall Passing"})
top_schools.head(5)

,School Name,School Type,Total Student Count,Total Budget ($),Per-Student Budget ($),Average Math Score,Average Reading Score,Number of Students Passing Math,Passing Math (%),Number of Students Passing Reading,Passing Reading (%),Number of Students Passing Overall,% Overall Passing
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,1749,94.133477,1803,97.039828,1697,91.334769
12,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,1525,93.272171,1591,97.308869,1487,90.948012
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,1371,93.392371,1426,97.138965,1330,90.599455
13,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,2143,93.867718,2204,96.539641,2068,90.582567
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,910,94.594595,923,95.945946,871,90.540541


## Lowest-Performing Schools by Percentage of Overall Passing

In [39]:
#organizing the summary data frame by 'total passing (%)', in ascending order.
bottom_schools = school_summary_final_copy.sort_values(by="Total Passing (%)",ascending=True)
#renaming column, to conform with challenge requirements
bottom_schools = bottom_schools.rename(columns={"Total Passing (%)": "% Overall Passing"})
bottom_schools.head(5)

,School Name,School Type,Total Student Count,Total Budget ($),Per-Student Budget ($),Average Math Score,Average Reading Score,Number of Students Passing Math,Passing Math (%),Number of Students Passing Reading,Passing Reading (%),Number of Students Passing Overall,% Overall Passing
10,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,2654,66.366592,3208,80.220055,2119,52.988247
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,1946,65.988471,2381,80.739234,1569,53.204476
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,1916,65.683922,2372,81.316421,1561,53.513884
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,3094,66.752967,3748,80.862999,2481,53.527508
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,3145,66.057551,3867,81.222432,2549,53.539172


## Math Scores by Grade

In [69]:
#average math scores per grade, by school

# Using the code provided in the assignment documentation to separate the data by grade
ninth_graders = students_df[(students_df["grade"] == "9th")]
tenth_graders = students_df[(students_df["grade"] == "10th")]
eleventh_graders = students_df[(students_df["grade"] == "11th")]
twelfth_graders = students_df[(students_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_grader_math_scores = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_grader_math_scores = eleventh_graders.groupby("school_name")["math_score"].mean()
twelfth_grader_math_scores = twelfth_graders.groupby("school_name")["math_score"].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.merge(ninth_grade_math_scores, tenth_grader_math_scores, on="school_name")
#renaming columns to rectify pd.merge() output. When merging to dataframes with shared column name "math_score", n\
#the new merged dataframe alters the names to "math_score_x" and "math_score_y".
math_scores_by_grade = math_scores_by_grade.rename(columns={"math_score_x": "9th Grade", "math_score_y": "10th Grade"})
math_scores_by_grade = pd.merge(math_scores_by_grade, eleventh_grader_math_scores, on="school_name")
math_scores_by_grade = math_scores_by_grade.rename(columns={"math_score": "11th Grade"})
math_scores_by_grade = pd.merge(math_scores_by_grade, twelfth_grader_math_scores, on="school_name")
math_scores_by_grade = math_scores_by_grade.rename(columns={"math_score": "12th Grade"})

# Minor data wrangling
math_scores_by_grade.index.name = None

#using mapping to udpate the formatting of series in dataframe (12)
math_scores_by_grade["9th Grade"] = math_scores_by_grade["9th Grade"].map("{:,.2f}".format)
math_scores_by_grade["10th Grade"] = math_scores_by_grade["10th Grade"].map("{:,.2f}".format)
math_scores_by_grade["11th Grade"] = math_scores_by_grade["11th Grade"].map("{:,.2f}".format)
math_scores_by_grade["12th Grade"] = math_scores_by_grade["12th Grade"].map("{:,.2f}".format)

# Display the DataFrame
math_scores_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Scores by Grade

In [73]:
#average reading scores by grade level, per school

# Using the code provided in the assignment documentation to separate the data by grade
ninth_graders = students_df[(students_df["grade"] == "9th")]
tenth_graders = students_df[(students_df["grade"] == "10th")]
eleventh_graders = students_df[(students_df["grade"] == "11th")]
twelfth_graders = students_df[(students_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_read_scores = ninth_graders.groupby("school_name")["reading_score"].mean()
tenth_grader_read_scores = tenth_graders.groupby("school_name")["reading_score"].mean()
eleventh_grader_read_scores = eleventh_graders.groupby("school_name")["reading_score"].mean()
twelfth_grader_read_scores = twelfth_graders.groupby("school_name")["reading_score"].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
read_scores_by_grade = pd.merge(ninth_grade_read_scores, tenth_grader_read_scores, on="school_name")
#renaming columns to rectify pd.merge() output. When merging to dataframes with shared column name "math_score", n\
#the new merged dataframe alters the names to "math_score_x" and "math_score_y".
read_scores_by_grade = read_scores_by_grade.rename(columns={"reading_score_x": "9th Grade", "reading_score_y": "10th Grade"})
read_scores_by_grade = pd.merge(read_scores_by_grade, eleventh_grader_read_scores, on="school_name")
read_scores_by_grade = read_scores_by_grade.rename(columns={"reading_score": "11th Grade"})
read_scores_by_grade = pd.merge(read_scores_by_grade, twelfth_grader_read_scores, on="school_name")
read_scores_by_grade = read_scores_by_grade.rename(columns={"reading_score": "12th Grade"})

# Minor data wrangling
math_scores_by_grade.index.name = None

#using mapping to udpate the formatting of series in dataframe (12)
read_scores_by_grade["9th Grade"] = read_scores_by_grade["9th Grade"].map("{:,.2f}".format)
read_scores_by_grade["10th Grade"] = read_scores_by_grade["10th Grade"].map("{:,.2f}".format)
read_scores_by_grade["11th Grade"] = read_scores_by_grade["11th Grade"].map("{:,.2f}".format)
read_scores_by_grade["12th Grade"] = read_scores_by_grade["12th Grade"].map("{:,.2f}".format)

# Display the DataFrame
read_scores_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [42]:
#Scores by school spending (per student)

#creating bins and lables. code provided in challenge documentation
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

#adding a new column to the overall summary dataframe, populating with bins (13)
school_summary_final_copy["Spending Ranges (Per Student)"] = pd.cut(school_summary_final_copy["Per-Student Budget ($)"],
                                             bins=spending_bins,labels=labels,include_lowest=True)

#calculating the mean of each group. code provided in challenge documentation
spending_math_scores = school_summary_final_copy.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_summary_final_copy.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_summary_final_copy.groupby(["Spending Ranges (Per Student)"])["Passing Math (%)"].mean()
spending_passing_reading = school_summary_final_copy.groupby(["Spending Ranges (Per Student)"])["Passing Reading (%)"].mean()
overall_passing_spending = school_summary_final_copy.groupby(["Spending Ranges (Per Student)"])["Total Passing (%)"].mean()


#merging data into single dataframe
spending_summary = pd.merge(spending_math_scores, spending_reading_scores,
                            on="Spending Ranges (Per Student)")

spending_summary = pd.merge(spending_summary, spending_passing_math, 
                            on="Spending Ranges (Per Student)")

spending_summary = pd.merge(spending_summary, spending_passing_reading, 
                            on="Spending Ranges (Per Student)")

spending_summary = pd.merge(spending_summary, overall_passing_spending, 
                            on="Spending Ranges (Per Student)")

#formatting
spending_summary["Average Math Score"] = spending_summary["Average Math Score"].map("{:,.2f}".format)
spending_summary["Average Reading Score"] = spending_summary["Average Reading Score"].map("{:,.2f}".format) 
spending_summary["Passing Math (%)"] = spending_summary["Passing Math (%)"].map("{:,.2f}%".format) 
spending_summary["Passing Reading (%)"] = spending_summary["Passing Reading (%)"].map("{:,.2f}%".format)
spending_summary["Total Passing (%)"] = spending_summary["Total Passing (%)"].map("{:,.2f}%".format) 

spending_summary

,Average Math Score,Average Reading Score,Passing Math (%),Passing Reading (%),Total Passing (%)
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$630-645,78.52,81.62,73.48%,84.39%,62.86%
$645-680,77.00,81.03,66.16%,81.13%,53.53%


## Scores by School Size

In [43]:
#Scores by School Size
#creating bins and lables. code provided in challenge documentation
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#adding a new column to the overall summary dataframe, populating with bins (13)
school_summary_final_copy["School Size"] = pd.cut(school_summary_final_copy["Total Student Count"],
                                             bins=size_bins,labels=labels,include_lowest=True)

#calculating the mean of each group. code provided in challenge documentation
size_math_scores = school_summary_final_copy.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = school_summary_final_copy.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = school_summary_final_copy.groupby(["School Size"])["Passing Math (%)"].mean()
size_passing_reading = school_summary_final_copy.groupby(["School Size"])["Passing Reading (%)"].mean()
overall_passing_size = school_summary_final_copy.groupby(["School Size"])["Total Passing (%)"].mean()

#merging data into single dataframe
size_summary = pd.merge(size_math_scores, size_reading_scores,
                            on="School Size")

size_summary = pd.merge(size_summary, size_passing_math, 
                            on="School Size")

size_summary = pd.merge(size_summary, size_passing_reading, 
                            on="School Size")

size_summary = pd.merge(size_summary, overall_passing_size, 
                            on="School Size")

#formatting
size_summary["Average Math Score"] = size_summary["Average Math Score"].map("{:,.2f}".format)
size_summary["Average Reading Score"] = size_summary["Average Reading Score"].map("{:,.2f}".format) 
size_summary["Passing Math (%)"] = size_summary["Passing Math (%)"].map("{:,.2f}%".format) 
size_summary["Passing Reading (%)"] = size_summary["Passing Reading (%)"].map("{:,.2f}%".format)
size_summary["Total Passing (%)"] = size_summary["Total Passing (%)"].map("{:,.2f}%".format) 


size_summary

,Average Math Score,Average Reading Score,Passing Math (%),Passing Reading (%),Total Passing (%)
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


## Scores by School Type

In [44]:
#code block structure taken from module challenge documentation/ starter-code, per instructions
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["Passing Math (%)"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["Passing Reading (%)"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["Total Passing (%)"].mean()


# combine groupby objects into a single DataFrame called `type_summary` (15)
type_summary = pd.DataFrame(pd.concat([average_math_score_by_type,average_reading_score_by_type,
                         average_percent_passing_math_by_type, average_percent_passing_reading_by_type,
                         average_percent_overall_passing_by_type], axis=1))

#formatting
type_summary["Average Math Score"] = type_summary["Average Math Score"].map("{:,.2f}".format)
type_summary["Average Reading Score"] = type_summary["Average Reading Score"].map("{:,.2f}".format) 
type_summary["Passing Math (%)"] = type_summary["Passing Math (%)"].map("{:,.2f}%".format) 
type_summary["Passing Reading (%)"] = type_summary["Passing Reading (%)"].map("{:,.2f}%".format)
type_summary["Total Passing (%)"] = type_summary["Total Passing (%)"].map("{:,.2f}%".format) 

type_summary

,Average Math Score,Average Reading Score,Passing Math (%),Passing Reading (%),Total Passing (%)
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
